In [8]:
import pandas as pd

file_path = r"C:\Users\HR\Desktop\Workspace\데이터톤\마케팅\merged_data.csv"
df = pd.read_csv(file_path)

print(df.head())


                  event_time event_type  product_id          category_id  \
0  2019-10-01 00:00:00+00:00       cart     5773203  1487580005134238553   
1  2019-10-01 00:00:03+00:00       cart     5773353  1487580005134238553   
2  2019-10-01 00:00:07+00:00       cart     5881589  2151191071051219817   
3  2019-10-01 00:00:07+00:00       cart     5723490  1487580005134238553   
4  2019-10-01 00:00:15+00:00       cart     5881449  1487580013522845895   

    brand  price    user_id                          user_session  \
0  runail   2.62  463240011  26dd6e6e-4dac-4778-8d2c-92e149dab885   
1  runail   2.62  463240011  26dd6e6e-4dac-4778-8d2c-92e149dab885   
2  lovely  13.48  429681830  49e8d843-adf3-428b-a2c3-fe8bc6a307c9   
3  runail   2.62  463240011  26dd6e6e-4dac-4778-8d2c-92e149dab885   
4  lovely   0.56  429681830  49e8d843-adf3-428b-a2c3-fe8bc6a307c9   

           event_time_moscow  event_month  \
0  2019-10-01 03:00:00+03:00         10.0   
1  2019-10-01 03:00:03+03:00         1

In [22]:
import pandas as pd

# ✅ 컬럼별 결측치 개수와 비율 계산
missing_summary = pd.DataFrame({
    'missing_count': df.isnull().sum(),
    'missing_ratio(%)': (df.isnull().sum() / len(df)) * 100
})

# ✅ 결측치가 있는 컬럼만 필터링 & 비율 기준 정렬
missing_summary = missing_summary[missing_summary['missing_count'] > 0].sort_values(by='missing_count', ascending=False)

print("✅ 결측치 현황 요약:")
print(missing_summary)



✅ 결측치 현황 요약:
             missing_count  missing_ratio(%)
brand              8755357         42.320717
event_month          11933          0.057680


In [23]:
df = df.dropna(subset=['event_month'])
print(df['event_month'].isna().sum())  # ✅ 0이 나와야 함


0


In [24]:
import pandas as pd

# CSV 불러오기
file_path = r"C:\Users\HR\Desktop\Workspace\데이터톤\마케팅\merged_data.csv"
df = pd.read_csv(file_path, encoding='utf-8-sig')

# ✅ 1. 기본 확인
print(df.info())
print("카테고리 개수:", df['category_id'].nunique())
print("category_id NULL 개수:", df['category_id'].isnull().sum())

# ✅ 2. 카테고리별 데이터 건수 TOP 10
top_categories = df['category_id'].value_counts().head(10)
print("\nTop 10 Categories by Count:\n", top_categories)

# ✅ 3. 카테고리별 매출 합계 TOP 10
if 'price' in df.columns:
    cat_revenue = df.groupby('category_id')['price'].sum().sort_values(ascending=False).head(10)
    print("\nTop 10 Categories by Revenue:\n", cat_revenue)

# ✅ 4. 이벤트 유형별 카테고리 분석
event_top_cat = df.groupby(['event_type', 'category_id']).size().reset_index(name='count')
event_top_cat = event_top_cat.sort_values(['event_type', 'count'], ascending=[True, False])

# 구매 이벤트에서 상위 10개 카테고리
purchase_top10 = event_top_cat[event_top_cat['event_type'] == 'purchase'].head(10)
print("\nPurchase Top 10 Categories:\n", purchase_top10)

# ✅ 5. 월별 카테고리 변화 (month 컬럼 존재할 때)
if 'month' in df.columns:
    monthly_cat = df.groupby(['month', 'category_id']).size().reset_index(name='count')
    monthly_pivot = monthly_cat.pivot(index='month', columns='category_id', values='count').fillna(0)
    print("\nMonthly Pivot Table (Top Rows):\n", monthly_pivot.head())

# ✅ Tableau에서 사용할 수 있도록 CSV 저장
top_categories.to_csv("top10_categories_count.csv", header=True)
if 'price' in df.columns:
    cat_revenue.to_csv("top10_categories_revenue.csv", header=True)
purchase_top10.to_csv("purchase_top10_categories.csv", index=False)
event_top_cat.to_csv("event_category_analysis.csv", index=False)
if 'month' in df.columns:
    monthly_cat.to_csv("monthly_category_trend.csv", index=False)

print("\n[CSV 파일 생성 완료]:")
print("- top10_categories_count.csv")
print("- top10_categories_revenue.csv (매출)")
print("- purchase_top10_categories.csv (구매 TOP 10)")
print("- event_category_analysis.csv (이벤트별 카테고리 분석)")
print("- monthly_category_trend.csv (월별 카테고리 변화)")



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20688111 entries, 0 to 20688110
Data columns (total 11 columns):
 #   Column             Dtype  
---  ------             -----  
 0   event_time         object 
 1   event_type         object 
 2   product_id         int64  
 3   category_id        int64  
 4   brand              object 
 5   price              float64
 6   user_id            int64  
 7   user_session       object 
 8   event_time_moscow  object 
 9   event_month        float64
 10  user_key           object 
dtypes: float64(2), int64(3), object(6)
memory usage: 1.7+ GB
None
카테고리 개수: 525
category_id NULL 개수: 0

Top 10 Categories by Count:
 category_id
1487580007675986893    1062619
1487580005595612013     789423
1487580005092295511     770878
1487580005671109489     680617
1487580006317032337     646659
1602943681873052386     640727
1487580005268456287     448730
1487580013841613016     397879
1487580005134238553     397102
1487580008246412266     375079
Name: count, d

In [25]:
import pandas as pd

# 1. CSV 파일 로드
file_path = r"C:\Users\HR\Desktop\Workspace\데이터톤\마케팅\merged_data.csv"
df = pd.read_csv(file_path, encoding='utf-8-sig')

#  1. 카테고리별 등장 빈도 TOP 10
top_cat_count = df['category_id'].value_counts().head(10)
print("Top 10 Categories by Event Count:\n", top_cat_count)

#  2. 카테고리별 매출 합계 TOP 10
cat_revenue = df.groupby('category_id')['price'].sum().sort_values(ascending=False).head(10)
print("\nTop 10 Categories by Revenue:\n", cat_revenue)

#  3. 이벤트 유형별 카테고리 분석 (구매 TOP 10)
event_cat = df.groupby(['event_type', 'category_id']).size().reset_index(name='count')
purchase_top10 = event_cat[event_cat['event_type'] == 'purchase'].sort_values('count', ascending=False).head(10)
print("\nPurchase Top 10:\n", purchase_top10)

#  4. 전환율 계산 (purchase/view 비율)
event_pivot = event_cat.pivot(index='category_id', columns='event_type', values='count').fillna(0)
event_pivot['conversion_rate'] = event_pivot['purchase'] / event_pivot['view']
event_pivot.replace([float('inf'), -float('inf')], 0, inplace=True)
top_conversion = event_pivot.sort_values('conversion_rate', ascending=False).head(10)
print("\nTop 10 Conversion Rate:\n", top_conversion)

#  5. 월별 카테고리 트렌드 데이터 준비 (Tableau에서 활용 가능)
monthly_cat = df.groupby(['event_month', 'category_id']).size().reset_index(name='count')

#  결과를 CSV로 저장 (Tableau용)
top_cat_count.to_csv('top10_category_count.csv', header=True)
cat_revenue.to_csv('top10_category_revenue.csv', header=True)
purchase_top10.to_csv('purchase_top10.csv', index=False)
event_pivot.reset_index().to_csv('category_conversion.csv', index=False)
monthly_cat.to_csv('monthly_category_trend.csv', index=False)

print("\n[CSV 파일 저장 완료] Tableau에서 사용 가능:")
print("- top10_category_count.csv")
print("- top10_category_revenue.csv")
print("- purchase_top10.csv")
print("- category_conversion.csv")
print("- monthly_category_trend.csv")


Top 10 Categories by Event Count:
 category_id
1487580007675986893    1062619
1487580005595612013     789423
1487580005092295511     770878
1487580005671109489     680617
1487580006317032337     646659
1602943681873052386     640727
1487580005268456287     448730
1487580013841613016     397879
1487580005134238553     397102
1487580008246412266     375079
Name: count, dtype: int64

Top 10 Categories by Revenue:
 category_id
1487580006300255120    32620834.09
1487580013950664926     8223231.83
1487580006350586771     7140877.60
1487580005092295511     6859485.84
1487580009051717646     6310745.21
2193074740619379535     4960254.99
1487580005268456287     3755005.92
1487580008246412266     3589102.28
1602943681873052386     3297211.10
1487580006317032337     2975730.80
Name: price, dtype: float64

Purchase Top 10:
     event_type          category_id  count
563   purchase  1487580007675986893  80137
519   purchase  1487580006317032337  50555
484   purchase  1487580005092295511  44870
498 

# 특징:

이 카테고리들은 노출 대비 구매 효율이 매우 높음

광고비/추천 상품에서 **ROI(투자 대비 효율)**가 매우 높을 가능성

구매 건수는 TOP10에 없지만, 핵심 타겟 카테고리로 활용 가능

In [26]:
# purchase 데이터만 필터링
purchase_df = df[df['event_type'] == 'purchase']

# 월별 카테고리별 구매 건수 집계
monthly_cat = purchase_df.groupby(['event_month', 'category_id']).size().reset_index(name='count')

# 각 월별 상위 5개 카테고리 뽑기
top5_per_month = (
    monthly_cat.groupby('event_month')
    .apply(lambda x: x.nlargest(5, 'count'), include_groups=False)
    .reset_index(drop=True)
)

print(top5_per_month)

top5_per_month.to_csv("monthly_top5_categories.csv", index=False)


            category_id  count
0   1487580007675986893  10994
1   1487580005595612013  10832
2   1487580005671109489  10800
3   1487580005092295511  10383
4   1487580006317032337  10129
5   1487580007675986893  13293
6   1487580006317032337   9632
7   1487580005092295511   8536
8   1487580005595612013   7339
9   1602943681873052386   7093
10  1487580007675986893  14477
11  1487580006317032337  10407
12  1487580005671109489   9895
13  1487580005092295511   7885
14  1487580009286598681   6158
15  1487580007675986893  25328
16  1487580005595612013  13730
17  1487580006317032337  12530
18  1487580005092295511  11340
19  1602943681873052386  10067
20  1487580007675986893  16024
21  1487580006317032337   7841
22  1487580005595612013   6954
23  1487580005092295511   6696
24  1487580005671109489   6030


In [27]:
import pandas as pd

# CSV 불러오기
file_path = r"C:\Users\HR\Desktop\Workspace\데이터톤\마케팅\merged_data.csv"
df = pd.read_csv(file_path, encoding='utf-8-sig')

#  1. 카테고리별 이벤트별 count 집계
event_summary = df.groupby(['category_id', 'event_type']).size().unstack(fill_value=0)

#  2. 필요한 컬럼만 정리
# 없는 컬럼 대비 대비 → get() 사용
event_summary['view_count'] = event_summary.get('view', 0)
event_summary['cart_count'] = event_summary.get('cart', 0)
event_summary['purchase_count'] = event_summary.get('purchase', 0)

#  3. 단계별 전환율 계산
event_summary['view_to_cart'] = (event_summary['cart_count'] / event_summary['view_count']).replace([float('inf'), -float('inf')], 0).fillna(0)
event_summary['cart_to_purchase'] = (event_summary['purchase_count'] / event_summary['cart_count']).replace([float('inf'), -float('inf')], 0).fillna(0)
event_summary['purchase_rate'] = (event_summary['purchase_count'] / event_summary['view_count']).replace([float('inf'), -float('inf')], 0).fillna(0)

#  4. 결과 정렬 (전체 전환율 기준 상위 10)
conversion_top10 = event_summary.sort_values('purchase_rate', ascending=False).head(10)

print("\n카테고리별 전환율 분석 (상위 10):\n", conversion_top10[['view_count', 'cart_count', 'purchase_count', 'view_to_cart', 'cart_to_purchase', 'purchase_rate']])

#  5. CSV 저장 (Tableau용)
conversion_top10[['view_count', 'cart_count', 'purchase_count', 'view_to_cart', 'cart_to_purchase', 'purchase_rate']].to_csv("category_conversion_analysis.csv")



카테고리별 전환율 분석 (상위 10):
 event_type           view_count  cart_count  purchase_count  view_to_cart  \
category_id                                                                 
1487580011476025461         977        1668             717      1.707267   
1487580007592100809        4454        9651            3001      2.166816   
1487580009622143014        2138        6216            1429      2.907390   
1487580010821714008        9524       16139            4841      1.694561   
2055161088059638328        8442       11516            4251      1.364132   
1487580009286598681       69864      203202           33862      2.908537   
1487580011425693811        4622        6898            2209      1.492428   
1487580010796548183       18284       29043            8610      1.588438   
1487580006509970331       13177       16418            6063      1.245959   
1487580009311764506       28118       68032           12596      2.419518   

event_type           cart_to_purchase  purchase_rat

# ✅ 결과 해석
    1. view_to_cart 비율 이상 현상
    예시: 1487580011476025461 → view_to_cart = 1.70
    → 장바구니 담은 건수가 조회 수보다 많음
    → 동일 사용자가 한 상품을 여러 번 카트에 담는 행동 가능성 (또는 이벤트 중복 기록)

    모든 카테고리에서 view_to_cart가 1 이상 → "조회 대비 장바구니 이벤트가 더 많다"
        → 의미: 카트에 담기 이벤트가 반복적으로 기록되는 데이터 구조 → 실제 고객 행동을 반영하는 방식 주의 필요

    2. cart_to_purchase 비율 (전환 효율성)
    가장 높은 카테고리:
    1487580006509970331 → 0.369, 2055161088059638328 → 0.369
    → 장바구니 담으면 약 36%가 구매로 이어짐

    가장 낮은 카테고리:
    1487580009286598681 → 0.166
    → 장바구니까지는 많이 담지만, 구매로는 잘 이어지지 않음 → 가격 or 품절 이슈 가능성

In [28]:
import pandas as pd

# CSV 불러오기
file_path = r"C:\Users\HR\Desktop\Workspace\데이터톤\마케팅\merged_data.csv"
df = pd.read_csv(file_path, encoding='utf-8-sig')

# ✅ 1. 카테고리별 이벤트 집계
event_summary = df.groupby(['category_id', 'event_type']).size().unstack(fill_value=0)

# ✅ 2. 필요한 컬럼 준비
event_summary['cart_count'] = event_summary.get('cart', 0)
event_summary['remove_count'] = event_summary.get('remove_from_cart', 0)
event_summary['purchase_count'] = event_summary.get('purchase', 0)

# ✅ 3. 이탈률 계산
event_summary['cart_remove_rate'] = (event_summary['remove_count'] / event_summary['cart_count']).replace([float('inf'), -float('inf')], 0).fillna(0)
event_summary['purchase_rate_from_cart'] = (event_summary['purchase_count'] / event_summary['cart_count']).replace([float('inf'), -float('inf')], 0).fillna(0)

# ✅ 4. 결과 정렬 (이탈률 상위 10)
high_abandon = event_summary.sort_values('cart_remove_rate', ascending=False).head(10)

# ✅ 5. 결과 확인
print("\n장바구니 이탈률 상위 10 카테고리:\n", high_abandon[['cart_count', 'remove_count', 'purchase_count', 'cart_remove_rate', 'purchase_rate_from_cart']])

# ✅ 6. CSV 저장 (Tableau용)
high_abandon[['cart_count', 'remove_count', 'purchase_count', 'cart_remove_rate', 'purchase_rate_from_cart']].to_csv("category_cart_abandonment.csv")



장바구니 이탈률 상위 10 카테고리:
 event_type           cart_count  remove_count  purchase_count  \
category_id                                                     
1487580005998265217           7            59               3   
1547480590851244887          42            84               4   
1791442703716712702           4             7               0   
1487580011828347011           7            10               0   
1487580005343953762         382           461              57   
1487580005897601916        3908          4369             972   
1487580013933887709          20            22               4   
1487580005067129686       10745         11124            2273   
1487580008128971748          38            39               5   
1487580010360340546          45            46               6   

event_type           cart_remove_rate  purchase_rate_from_cart  
category_id                                                     
1487580005998265217          8.428571                 0.428571  
1

# ✅ 결과 해석
    1. 이탈률(cart_remove_rate) 비정상적 패턴
    예: 1487580005998265217 → 8.42
    → 이탈 이벤트가 카트 이벤트보다 8배 많음
    → 두 가지 가능성:

    동일 아이템을 여러 번 장바구니에 담고 여러 번 제거하는 행동

    이벤트 데이터 로깅 방식 문제 (이벤트가 누적 기록됨)
    → 데이터 클린징 시 중복 제거 또는 세션 기준 집계 고려 필요.

    2. 이탈률 > 1 대부분
    1547480590851244887 → 2.0

    1487580005343953762 → 1.20
    → remove 이벤트가 cart보다 많음
    → 일반적으로는 불가능 → 이탈 분석할 때 절대 수치보다는 비율 비교가 아닌 순서 분석 필요.

    3. 구매율 (purchase/cart)
    대부분 낮음 (0.09 ~ 0.24)
    → 장바구니 이탈률 높은 카테고리는 구매로 이어지는 비율도 낮음
    → 원인:

    비싼 상품 (고객 망설임)

    재고 부족

    UX 문제(배송비, 할인 조건)

In [29]:
import pandas as pd

# ✅ 0. CSV 로드
file_path = r"C:\Users\HR\Desktop\Workspace\데이터톤\마케팅\merged_data.csv"
df = pd.read_csv(file_path, encoding='utf-8-sig')

# -------------------------------------------------
# ✅ 1. 브랜드별 이탈률 분석
brand_summary = df.groupby(['brand', 'event_type']).size().unstack(fill_value=0)

# 필요한 컬럼 준비
brand_summary['cart_count'] = brand_summary.get('cart', 0)
brand_summary['remove_count'] = brand_summary.get('remove_from_cart', 0)
brand_summary['purchase_count'] = brand_summary.get('purchase', 0)

# 이탈률 계산
brand_summary['cart_remove_rate'] = (brand_summary['remove_count'] / brand_summary['cart_count']).replace([float('inf'), -float('inf')], 0).fillna(0)
brand_summary['purchase_rate_from_cart'] = (brand_summary['purchase_count'] / brand_summary['cart_count']).replace([float('inf'), -float('inf')], 0).fillna(0)

# 상위 10개 확인
brand_top10 = brand_summary.sort_values('cart_remove_rate', ascending=False).head(10)
print("\n브랜드별 이탈률 상위 10:\n", brand_top10[['cart_count', 'remove_count', 'purchase_count', 'cart_remove_rate', 'purchase_rate_from_cart']])

# CSV 저장
brand_summary[['cart_count', 'remove_count', 'purchase_count', 'cart_remove_rate', 'purchase_rate_from_cart']].reset_index().to_csv("brand_cart_abandonment.csv", index=False)

# -------------------------------------------------
# ✅ 2. 월별 이탈률 트렌드
# 월별 + 카테고리별 집계
monthly_event = df.groupby(['event_month', 'event_type', 'category_id']).size().unstack(fill_value=0)

# cart/remove/purchase 합계
monthly_event['cart_count'] = monthly_event.get('cart', 0)
monthly_event['remove_count'] = monthly_event.get('remove_from_cart', 0)
monthly_event['purchase_count'] = monthly_event.get('purchase', 0)

# 월별 평균 이탈률 & 구매율
monthly_event['cart_remove_rate'] = (monthly_event['remove_count'] / monthly_event['cart_count']).replace([float('inf'), -float('inf')], 0).fillna(0)
monthly_event['purchase_rate_from_cart'] = (monthly_event['purchase_count'] / monthly_event['cart_count']).replace([float('inf'), -float('inf')], 0).fillna(0)

monthly_avg = monthly_event.groupby('event_month')[['cart_remove_rate', 'purchase_rate_from_cart']].mean().reset_index()
print("\n월별 평균 이탈률 및 구매율:\n", monthly_avg)

# CSV 저장
monthly_avg.to_csv("monthly_cart_abandonment_trend.csv", index=False)

# -------------------------------------------------
# ✅ 3. 이탈률 vs 가격 상관관계 분석
# 카테고리별 평균 가격 계산
avg_price_per_category = df.groupby('category_id')['price'].mean().reset_index()

# 카테고리별 이탈률
category_summary = df.groupby(['category_id', 'event_type']).size().unstack(fill_value=0)
category_summary['cart_count'] = category_summary.get('cart', 0)
category_summary['remove_count'] = category_summary.get('remove_from_cart', 0)
category_summary['cart_remove_rate'] = (category_summary['remove_count'] / category_summary['cart_count']).replace([float('inf'), -float('inf')], 0).fillna(0)

# price와 join
category_summary = category_summary.reset_index().merge(avg_price_per_category, on='category_id', how='left')

# 상관계수 계산
correlation = category_summary['cart_remove_rate'].corr(category_summary['price'])
print(f"\n카테고리별 이탈률 vs 평균 가격 상관계수: {correlation:.4f}")

# CSV 저장 (Tableau용)
category_summary[['category_id', 'cart_remove_rate', 'price']].to_csv("cart_abandonment_vs_price.csv", index=False)



브랜드별 이탈률 상위 10:
 event_type  cart_count  remove_count  purchase_count  cart_remove_rate  \
brand                                                                    
note               470           541              50          1.151064   
busch                8             8               0          1.000000   
vichy              183           171              26          0.934426   
frozen              57            53               7          0.929825   
lianail          35398         32716            6642          0.924233   
dancoly             11            10               0          0.909091   
masura          247018        222714           49751          0.901610   
uskusi           29160         26091            5708          0.894753   
tertio            1755          1567             227          0.892877   
haruyama         55119         48690           11582          0.883361   

event_type  purchase_rate_from_cart  
brand                                
note             

# 결과 해석
    ✅ 1. 브랜드별 이탈률 분석
    특징:
    이탈률 상위 브랜드:
    note (1.15), busch (1.00), vichy (0.93), lianail (0.92), masura (0.90)

    이탈률이 1.0 이상인 브랜드는 remove 이벤트가 cart보다 많음 → 이벤트 중복 가능성 or 사용자 행동(담았다가 빼기 반복).

    **구매율(purchase/cart)**도 낮음:

    note → 10%

    masura → 20%

    haruyama → 21% (상대적으로 양호)

    인사이트:
    이탈률 높고 구매율 낮은 브랜드 → 프로모션 강화 대상

    이탈률 높지만 구매율 높은 브랜드 → 고객이 망설이다 결국 사는 경우 → 가격 민감성 가능

    ✅ 2. 월별 이탈률 트렌드
    결과:
    0.0으로 표시 → 월별 컬럼 처리 문제 발생

    원인: event_month가 float이거나 NaN, 혹은 집계할 이벤트 없음

    해결: df['event_month'] 값 확인 필요 (df['event_time']에서 추출 가능)

    ✅ 3. 가격 vs 이탈률 상관관계
    상관계수: -0.0368 → 거의 0 (상관관계 없음)
    
    해석:

    가격이 비싸다고 이탈률이 높지도 않고
    
    저가 상품이라고 이탈률이 낮지도 않음
    → 즉, 가격보다는 브랜드나 카테고리 특성이 이탈에 영향.

In [31]:
import pandas as pd

# ✅ 데이터 로드
file_path = r"C:\Users\HR\Desktop\Workspace\데이터톤\마케팅\merged_data.csv"
df = pd.read_csv(file_path, encoding='utf-8-sig')

# ✅ 1. event_month 컬럼 생성 (정확히)
df['event_time'] = pd.to_datetime(df['event_time'], errors='coerce')  # 시간 변환
df['event_month'] = df['event_time'].dt.month  # 월 추출 (1~12)

# ✅ 2. 월별 + 카테고리별 집계
monthly_event = df.groupby(['event_month', 'category_id', 'event_type']).size().unstack(fill_value=0)

# ✅ 3. cart/remove/purchase 컬럼 준비
monthly_event['cart_count'] = monthly_event.get('cart', 0)
monthly_event['remove_count'] = monthly_event.get('remove_from_cart', 0)
monthly_event['purchase_count'] = monthly_event.get('purchase', 0)

# ✅ 4. 이탈률 & 구매율 계산
monthly_event['cart_remove_rate'] = (monthly_event['remove_count'] / monthly_event['cart_count']).replace([float('inf'), -float('inf')], 0).fillna(0)
monthly_event['purchase_rate_from_cart'] = (monthly_event['purchase_count'] / monthly_event['cart_count']).replace([float('inf'), -float('inf')], 0).fillna(0)

# ✅ 5. 월별 평균 이탈률 & 구매율 (모든 카테고리 평균)
monthly_avg = monthly_event.groupby('event_month')[['cart_remove_rate', 'purchase_rate_from_cart']].mean().reset_index()

# ✅ 결과 확인
print("\n월별 평균 이탈률 및 구매율:\n", monthly_avg)

# ✅ CSV 저장 (Tableau용)
monthly_avg.to_csv("monthly_cart_abandonment_trend.csv", index=False)



월별 평균 이탈률 및 구매율:
 event_type  event_month  cart_remove_rate  purchase_rate_from_cart
0                     1          0.619256                 0.185912
1                     2          0.614035                 0.171884
2                    10          0.494371                 0.162848
3                    11          0.728474                 0.219234
4                    12          0.641463                 0.192232


# 인사이트
    11월(0.728) → 이탈률 최고
    → 쇼핑 시즌(블랙프라이데이)과 연관 가능성

    고객이 많이 담았다가 제거 (가격 비교, 다른 프로모션 탐색)

    10월(0.494) → 이탈률 최저
    → 이 시기에는 장바구니 유지 비율이 상대적으로 높음

    구매율 트렌드

    11월 구매율도 최고(0.219) → 이탈도 많지만 구매도 활발

    연말(12월)도 구매율 양호 → 시즌 영향 확인 가능

    계절성 패턴

    10월: 안정적

    11~12월: 프로모션 영향 → 고객의 탐색 행위 증가

# ✅ 활용 포인트
    11월: 이탈률 높음 → 가격 경쟁력 확보 + 리마인드 메시지 강화

    10월: 충성 고객 유지 → 적극 리타겟팅

    12월: 연말 특수 활용 → 번들 할인/쿠폰 전략

